# 5 Min Binary Options Predictions on FX

# LSTM method (Long-term short-term memory)

## Import required libraries to train LSTM deep nerual network

In [12]:
import matplotlib
import numpy as np
import pandas as pd
import itertools
import sklearn
import keras
#import tensorflow
import time

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from matplotlib import pyplot as plt
from sklearn import preprocessing




import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from mpl_finance import candlestick_ohlc
import numpy as np
import urllib
import datetime as dt



print('Numpy version: ' + np.__version__)
print('Pandas version: ' + pd.__version__)
print('Matplotlib version: ' + matplotlib.__version__)
print('Sklearn version: ' + sklearn.__version__)
print('Keras version: ' + keras.__version__)
#print('Tensorflow-gpu version: ' + tensorflow.__version__)

ModuleNotFoundError: No module named 'mpl_finance'

### RSI function - Relative strength index

In [ ]:
def relative_strength(prices, n=14):

    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed >= 0].sum()/n
    down = -seed[seed < 0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1. + rs)

    for i in range(n, len(prices)):
        delta = deltas[i - 1]  # cause the diff is 1 shorter

        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n - 1) + upval)/n
        down = (down*(n - 1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1. + rs)

    return rsi

## Main feature enginering function

In [ ]:
def FeatureEnginering(MyRawData):


    MyRawData['Gmt time'] = pd.to_datetime(MyRawData['Gmt time'], format='%d.%m.%Y %H:%M:%S.000', utc=True)
    
    no_of_std = 2

    MyRawData['MA_30MA'] = MyRawData['Close'].rolling(window=30).mean()
    MyRawData['MA_30MA_std'] = MyRawData['Close'].rolling(window=30).std() 
    MyRawData['MA_30MA_BB_high'] = MyRawData['MA_30MA'] + no_of_std * MyRawData['MA_30MA_std']
    MyRawData['MA_30MA_BB_low'] = MyRawData['MA_30MA'] - no_of_std * MyRawData['MA_30MA_std']

    MyRawData['RSI'] =relative_strength(MyRawData['Close'])

    del MyRawData['Volume']
    del MyRawData['Gmt time']
    del MyRawData['MA_30MA_std']

    return MyRawData



# Function to plot candlesticks

In [ ]:
def graph_data_ohlc(dataset):
    fig = plt.figure()
    ax1 = plt.subplot2grid((1,1), (0,0))
    closep=dataset[:,[3]]
    highp=dataset[:,[1]]
    lowp=dataset[:,[2]]
    openp=dataset[:,[0]]
    date=range(len(closep))
    MA_30MA = dataset[:,[4]]
    MA_30MA_std = dataset[:,[5]]
    MA_30MA_BB_low = dataset[:,[6]]
    MA_30MA_BB_high = dataset[:,[7]]
    RSI = dataset[:,[8]]
    
    
    x = 0
    y = len(date)
    ohlc = []
    while x < y:
        append_me = date[x], openp[x], highp[x], lowp[x], closep[x]
        ohlc.append(append_me)
        x+=1
    candlestick_ohlc(ax1, ohlc, width=0.4, colorup='#77d879', colordown='#db3f3f')
    for label in ax1.xaxis.get_ticklabels():
        label.set_rotation(45)
    ax1.xaxis.set_major_locator(mticker.MaxNLocator(10))
    ax1.grid(True)
    plt.xlabel('Candle')
    plt.ylabel('Price')
    plt.title('Candlestick sample representation')
    plt.legend()
    
    ax1.plot(date, MA_30MA)
    ax1.plot(date, MA_30MA_BB_low)
    ax1.plot(date, MA_30MA_BB_high)
    
    plt.subplots_adjust(left=0.09, bottom=0.20, right=0.94, top=0.90, wspace=0.2, hspace=0)
    plt.show()

# Load the dataset

In [21]:
df = pd.read_csv('EURUSD_2010_2018.csv')


#### Raw dataset

In [22]:
df.head(5)

,Gmt time,Open,High,Low,Close,Volume
0,01.01.2010 00:00:00.000,1.43283,1.43283,1.43267,1.43267,74.3
1,01.01.2010 00:01:00.000,1.43268,1.43285,1.43268,1.43284,25.4
2,01.01.2010 00:02:00.000,1.43280,1.43280,1.43268,1.43280,30.9
3,01.01.2010 00:03:00.000,1.43280,1.43280,1.43268,1.43273,41.9
4,01.01.2010 00:04:00.000,1.43283,1.43283,1.43277,1.43277,53.8


#### Feature enginered dataset

In [23]:
data_set = FeatureEnginering(df)

NameError: name 'FeatureEnginering' is not defined

In [24]:
df.tail(5)

,Gmt time,Open,High,Low,Close,Volume
3363308,31.12.2018 21:56:00.000,1.14637,1.14642,1.14624,1.14642,94.97
3363309,31.12.2018 21:57:00.000,1.14642,1.14647,1.14640,1.14645,89.03
3363310,31.12.2018 21:58:00.000,1.14645,1.14674,1.14639,1.14672,96.00
3363311,31.12.2018 21:59:00.000,1.14671,1.14671,1.14635,1.14644,26.77
3363312,31.12.2018 22:00:00.000,1.14644,1.14644,1.14627,1.14627,5.25


In [25]:
data_set = data_set.values

NameError: name 'data_set' is not defined

In [26]:
#min_max_scaler = preprocessing.MinMaxScaler()
#data_set = min_max_scaler.fit_transform(data_set)

In [15]:
data_set

NameError: name 'data_set' is not defined

In [16]:
def convert_price_to_pips(conversion_array):
    #Open	High	Low 	Close	MA_30MA 	MA_30MA_BB_high 	MA_30MA_BB_low	RSI
    #Open-->Wicks up
    #High-->Wicks down
    #Low-->Body size
    #dClose-->Candle type {1,-1}
    #MA_30MA-->(Close - MA_30MA)
    #MA_30MA_BB_high-->(Close -  MA_30MA_BB_high)
    #MA_30MA_BB_low-->(Close - MA_30MA_BB_low)
    #RSI--> (RSI * 0.01)
    

    if conversion_array[3]>conversion_array[0]:
        candle_type=1
        wicks_up=conversion_array[1]-conversion_array[3]
        wicks_down=conversion_array[2]-conversion_array[0]
        body_size=conversion_array[3]-conversion_array[0]
    else:
        candle_type=-1
        wicks_up=conversion_array[1]-conversion_array[0]
        wicks_down=conversion_array[2]-conversion_array[3]
        body_size=conversion_array[1]-conversion_array[3]
    
    MA=conversion_array[4]-conversion_array[3]
    BB_high=conversion_array[5]-conversion_array[3]
    BB_high=conversion_array[6]-conversion_array[3]
    RSI=conversion_array[7]*0.01
    
    conversion_array[0]=candle_type
    conversion_array[1]=wicks_up
    conversion_array[2]=wicks_down
    conversion_array[3]=body_size
    conversion_array[4]=MA
    conversion_array[5]=BB_high
    conversion_array[6]=BB_high
    conversion_array[7]=RSI
    
    return conversion_array

In [17]:
def generator_pips(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            
            if data[rows[j] + delay][3] > data[rows[j]][3]:
                targets[j] = 1
            else:
                targets[j] = 0
        yield samples, targets

In [18]:

def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            
            #min_max_scaler = preprocessing.MinMaxScaler()
            new_samples=[]
            for h in data[indices]:
                new_samples.append(convert_price_to_pips(h)) 
        
            new_samples = np.array(new_samples)
            #new_samples = min_max_scaler.fit_transform(new_samples)
            
            samples[j] = data[indices]
            #samples[j] = new_samples
            
            if data[rows[j] + delay][3] > data[rows[j]][3]:
                targets[j] = 1
            else:
                targets[j] = 0
        yield samples, targets

In [19]:
lookback = 50
step = 1
delay = 60
batch_size = 500

train_gen = generator(data_set,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=2000000,
                      shuffle=False,
                      step=step, 
                      batch_size=batch_size)
val_gen = generator(data_set,
                    lookback=lookback,
                    delay=delay,
                    min_index=2000001,
                    max_index=3000000,
                    step=step,
                    batch_size=batch_size)
test_gen = generator(data_set,
                     lookback=lookback,
                     delay=delay,
                     min_index=3000001,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

# Liczba kroków pobierania danych z obiektu val_gen 
# zapewniających przetworzenie całego walidacyjnego zbioru danych.
val_steps = (3000000 - 2000001 - lookback) // batch_size

# Liczba kroków pobierania danych z obiektu test_gen, 
# przy której przetworzony zostanie cały testowy zbiór danych.
test_steps = (len(data_set) - 3000001 - lookback) // batch_size

NameError: name 'data_set' is not defined

In [20]:
def testooos():
    test = next(train_gen)
    graph_data_ohlc(test[0][0])

In [10]:
print(next(train_gen))

NameError: name 'train_gen' is not defined

import time
starttime=time.time()
while True:
    testooos()
    time.sleep(1.0 - ((time.time() - starttime) % 1.0))

In [3]:
###### from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()

#model = Sequential()
#model.add(layers.GRU(32,
   #                  dropout=0.2,
  #                   recurrent_dropout=0.2,
  #                   input_shape=(None, float_data.shape[-1])))
#model.add(layers.Dense(1))

model.add(layers.CuDNNLSTM(units = 50, return_sequences = True, input_shape = (None, data_set.shape[-1])))
model.add(layers.Dropout(0.2))

model.add(layers.CuDNNLSTM(units = 50, return_sequences = True))
model.add(layers.Dropout(0.2))

model.add(layers.CuDNNLSTM(units = 50, return_sequences = True))
model.add(layers.Dropout(0.2))

model.add(layers.CuDNNLSTM(units = 50))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(units = 1))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=20,
                              validation_data=val_gen,
                              validation_steps=val_steps)




Using TensorFlow backend.


NameError: name 'Sequential' is not defined

In [ ]:
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.figure()

plt.plot(epochs, loss, 'bo', label='Strata trenowania')
plt.plot(epochs, val_loss, 'b', label='Strata walidacji')
plt.title('Strata trenowania i walidacji')
plt.legend()

plt.show()

In [20]:
test_loss, test_acc = model.evaluate(test_gen)
print('Test accuracy:', test_acc)
predictions = model.predict_generator(test_gen)

ValueError: `steps=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `steps` or use the `keras.utils.Sequence` class.